In [ ]:
! pip install category_encoders

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from category_encoders import BinaryEncoder
from sklearn.dummy import DummyRegressor

In [ ]:
df = pd.read_csv(filepath_or_buffer='rome_listings_aftereda.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
## One Hot encoding for room_type
df = pd.get_dummies(df, columns=['room_type'], prefix='room_type')
df = pd.get_dummies(df, columns=['neighbourhood_cleansed'], prefix='neighbourhood')

In [ ]:
## convert boolean columns to int
bool_cols = df.columns[df.dtypes == 'bool']
df[bool_cols] = df[bool_cols].astype(int)

In [ ]:
binary_encoder = BinaryEncoder()
binary_encoder.fit_transform(df['property_type'])
# Create a DataFrame with the new columns
new_columns = pd.DataFrame(binary_encoder.transform(df['property_type']), columns=binary_encoder.get_feature_names_out())

# Concatenate the new columns with the original dataset
df = pd.concat([df, new_columns], axis=1)
df.drop(columns=['property_type'], inplace=True)

In [ ]:
price = df.pop('logprice')

In [ ]:
price.hist()

In [ ]:
df.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, price, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
columns_to_standardize = ['accommodates', 'bathrooms', 'bedrooms', 'beds']
scaler = StandardScaler()
X_train[columns_to_standardize] = scaler.fit_transform(X_train[columns_to_standardize])
X_test[columns_to_standardize] = scaler.transform(X_test[columns_to_standardize])

In [ ]:
# Step 3: Perform PCA
threshold = 0.95
pca = PCA().fit(X_train)

# Step 4: Calculate explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Step 5: Plot the cumulative explained variance
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.axhline(y=threshold, color='r', linestyle='-')
n_components = np.argmax(cumulative_variance >= threshold) + 1
plt.axvline(x=n_components, color='b', linestyle='--')

# Annotations
plt.annotate(f'{n_components} Components',
             xy=(n_components, threshold),
             xytext=(n_components + 2, threshold - 0.1),
             arrowprops=dict(facecolor='black', arrowstyle='->'),
             fontsize=10)
plt.text(n_components, 0.02, f'{n_components}', color='blue', ha='center', fontsize=12)
plt.scatter(n_components, threshold, color='red', zorder=5)

# Labels and title
plt.title('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid()
plt.show()

print(f"Number of components chosen: {n_components}")

In [ ]:
pca = PCA(n_components=n_components)
X_train_transform = pca.fit_transform(X_train)
X_test_transform = pca.transform(X_test)

In [ ]:
X_train_transform.shape

In [ ]:
# Linear Regression
param_grid = {}
scoring = make_scorer(mean_squared_error, greater_is_better=False)
model = LinearRegression()
grid_search_linreg = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=KFold(5, random_state=2, shuffle=True),
    n_jobs=-1
)
grid_search_linreg.fit(X_train_transform, y_train)
# print("Best parameters found: ", grid_search_linreg.best_params_)
print("Best mean squared error: ", -grid_search_linreg.best_score_)

In [ ]:
# Ridge Regression
alphas = np.logspace(-6, 6, 100, base=10)
param_grid = {'alpha': alphas}
scoring = make_scorer(mean_squared_error, greater_is_better=False)
model = Ridge()
grid_search_ridge = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=KFold(5, random_state=2, shuffle=True),
    n_jobs=-1
)
grid_search_ridge.fit(X_train, y_train)
print("Best parameters found: ", grid_search_ridge.best_params_)
print("Best mean squared error: ", -grid_search_ridge.best_score_)


In [ ]:
# Lasso
alphas = np.logspace(-6, 6, 100, base=10)
param_grid = {'alpha': alphas}
model = Lasso()
grid_search_lasso = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=KFold(5, random_state=2, shuffle=True),
    n_jobs=-1
)
grid_search_lasso.fit(X_train, y_train)
print("Best parameters found: ", grid_search_lasso.best_params_)
print("Best mean squared error: ", -grid_search_lasso.best_score_)

In [ ]:
# Decision Tree
param_grid = param_grid = {
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2'],
    'min_impurity_decrease': [0.0, 0.1, 0.2],
    'ccp_alpha': [0.0, 0.1, 0.2],
    'random_state': [2]  # Adjust this according to your needs
}
model = DecisionTreeRegressor()
grid_search_dt = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=KFold(5, random_state=2, shuffle=True),
    n_jobs=-1,
    verbose=1
)
grid_search_dt.fit(X_train, y_train)
print("Best parameters found: ", grid_search_dt.best_params_)
print("Best mean squared error: ", -grid_search_dt.best_score_)

In [ ]:
# Random Forest
param_grid = param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
model = RandomForestRegressor()
grid_search_rf = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=KFold(5, random_state=2, shuffle=True),
    n_jobs=-1,
    verbose=1
)
grid_search_rf.fit(X_train, y_train)
print("Best parameters found: ", grid_search_rf.best_params_)
print("Best mean squared error: ", -grid_search_rf.best_score_)

In [ ]:
# XGBoost
param_grid = param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
}
model = xgb.XGBRegressor(random_state=2)
grid_search_xgb = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    cv=KFold(5, random_state=2, shuffle=True),
    n_jobs=-1
)
grid_search_xgb.fit(X_train, y_train)
print("Best parameters found: ", grid_search_xgb.best_params_)
print("Best mean squared error: ", -grid_search_xgb.best_score_)

In [ ]:
dummy_reg_mean = DummyRegressor(strategy='mean')
dummy_reg_mean.fit(X_train, y_train)

In [ ]:
# Evaluate the best model
performance = {}
trained = {
    'Ridge': grid_search_ridge.best_estimator_,
    'Lasso': grid_search_lasso.best_estimator_,
    'DecisionTree': grid_search_dt.best_estimator_,
    'RandomForest': grid_search_rf.best_estimator_,
    'XGBoost': grid_search_xgb.best_estimator_,
    'Dummy_mean': dummy_reg_mean
}
for name, model in trained.items():
    predictions = model.predict(X_test)
    mse = mean_squared_error(np.exp(y_test), np.exp(predictions))
    rmse = np.sqrt(mse)
    r_squared = r2_score(np.exp(y_test), np.exp(predictions))
    performance[name] = mse
    print(f"{name} mean squared error: {mse}", f"root mean squared error: {rmse}", f"R^2: {r_squared}")

In [ ]:
model = grid_search_linreg.best_estimator_
name = "Linear Regression"
X_test_transform = pca.transform(X_test)
predictions = model.predict(X_test_transform)
mse = mean_squared_error(np.exp(y_test), np.exp(predictions))
rmse = np.sqrt(mse)
r_squared = r2_score(np.exp(y_test), np.exp(predictions))
performance[name] = mse
print(f"{name} mean squared error: {mse}", f"root mean squared error: {rmse}", f"R^2: {r_squared}")

In [ ]:
# Table of Results
performance = {}
trained = {
    'Ridge': grid_search_ridge.best_estimator_,
    'Lasso': grid_search_lasso.best_estimator_,
    'DecisionTree': grid_search_dt.best_estimator_,
    'RandomForest': grid_search_rf.best_estimator_,
    'XGBoost': grid_search_xgb.best_estimator_,
    'Dummy_mean': dummy_reg_mean
}

# Evaluate performance for each model in 'trained'
for name, model in trained.items():
    predictions = model.predict(X_test)
    mse = mean_squared_error(np.exp(y_test), np.exp(predictions))
    rmse = np.sqrt(mse)
    r_squared = r2_score(np.exp(y_test), np.exp(predictions))
    performance[name] = {'MSE': mse, 'RMSE': rmse, 'R^2': r_squared}
    print(f"{name} mean squared error: {mse}", f"root mean squared error: {rmse}", f"R^2: {r_squared}")

# Evaluate performance for Linear Regression model separately
model = grid_search_linreg.best_estimator_
name = "Linear Regression"
X_test_transform = pca.transform(X_test)
predictions = model.predict(X_test_transform)
mse = mean_squared_error(np.exp(y_test), np.exp(predictions))
rmse = np.sqrt(mse)
r_squared = r2_score(np.exp(y_test), np.exp(predictions))
performance[name] = {'MSE': mse, 'RMSE': rmse, 'R^2': r_squared}
print(f"{name} mean squared error: {mse}", f"root mean squared error: {rmse}", f"R^2: {r_squared}")

# Convert the performance dictionary to a DataFrame and display it
performance_df = pd.DataFrame(performance).T
print(performance_df)

In [ ]:
# Plotting MSE
plt.figure(figsize=(10, 5))
plt.bar(performance_df.index, performance_df['MSE'], color='skyblue')
plt.xlabel('Model')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Mean Squared Error (MSE) by Model')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Plotting R-squared
plt.figure(figsize=(10, 5))
plt.bar(performance_df.index, performance_df['R^2'], color='lightgreen')
plt.xlabel('Model')
plt.ylabel('R-squared (R^2)')
plt.title('R-squared (R^2) by Model')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
importances = grid_search_rf.best_estimator_.feature_importances_
feature_names = X_train.columns

# Step 6: Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_names, importances, color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance in RandomForestRegressor')
plt.show()


In [ ]:
importances = grid_search_rf.best_estimator_.feature_importances_
feature_names = X_train.columns

# Plot the feature importances
plt.figure(figsize=(12, 8))

colormap = plt.cm.viridis
bars = plt.barh(feature_names, importances, color=colormap(importances))
plt.xlabel('Feature Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.title('Feature Importance in RandomForestRegressor', fontsize=16)
plt.show()